# Sklearn

## sklearn.grid_search

документация: http://scikit-learn.org/stable/modules/grid_search.html

In [1]:
from sklearn import datasets, model_selection, linear_model, metrics
import numpy as np
import pandas as pd

### Генерация датасета

In [2]:
iris = datasets.load_iris() # Генерируем набор данных данные (данные ирис)

In [3]:
#Разбиваем данные на тестовые и обучающие
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(iris.data, iris.target, 
                                                                                     test_size = 0.3,random_state = 0)

### Задание модели

В качестве примера, мы будем использовать Стохастический градиентный спуск.


В обчном методе градиентного спука на каждом шаге итерации следующее приблежение получатеся из предыдущего вычитанием вектора градиента, умноженного на шаг $ \eta_t$:

$$ w^t = w^{t-1} - \eta_t \nabla(w^{t-1},X) $$

При этом выражение для градиента в матричной форме имеет вид: 

$$ \nabla_w Q(w,X) = \frac{2}{l}X^T(Xw-y) $$ 

Выражение для j-ой компоненты градиента, таким образом, содержит, суммирование по всем объектам обучающей выборки:

$$  $$



In [114]:

classifier = linear_model.SGDClassifier(random_state = 0,max_iter=1000)

### Генерация сетки

In [115]:
classifier.get_params().keys()

dict_keys(['alpha', 'average', 'class_weight', 'early_stopping', 'epsilon', 'eta0', 'fit_intercept', 'l1_ratio', 'learning_rate', 'loss', 'max_iter', 'n_iter', 'n_iter_no_change', 'n_jobs', 'penalty', 'power_t', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [116]:
parameters_grid = {
    'loss' : ['hinge', 'log', 'squared_hinge', 'squared_loss'],
    'penalty' : ['l1', 'l2'],
    'max_iter': range(5,10),
    'alpha' : np.linspace(0.0001, 0.001, num = 5),
}

In [120]:
cv = model_selection.StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state = 0)

### Подбор параметров и оценка качества

#### Grid search

In [121]:
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

In [122]:
%%time
grid_cv.fit(iris.data, iris.target)

CPU times: user 2.99 s, sys: 0 ns, total: 2.99 s
Wall time: 3.02 s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
       error_score='raise-deprecating',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=0, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss'], 'penalty': ['l1', 'l2'], 'max_iter': range(5, 10), 'alpha': array([0.0001 , 0.00032, 0.00055, 0.00078, 0.001  ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [123]:
grid_cv.best_estimator_

SGDClassifier(alpha=0.0007750000000000001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=9,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l1',
       power_t=0.5, random_state=0, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [124]:
print (grid_cv.best_score_)
print (grid_cv.best_params_)

0.87
{'alpha': 0.0007750000000000001, 'loss': 'hinge', 'max_iter': 9, 'penalty': 'l1'}


In [125]:
grid_cv.best_params_

{'alpha': 0.0007750000000000001,
 'loss': 'hinge',
 'max_iter': 9,
 'penalty': 'l1'}

#### Randomized grid search

In [126]:
randomized_grid_cv = model_selection.RandomizedSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, 
                                                   random_state = 0)

In [127]:
%%time
randomized_grid_cv.fit(train_data, train_labels)

CPU times: user 149 ms, sys: 207 µs, total: 149 ms
Wall time: 159 ms


RandomizedSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=0, test_size=0.2,
            train_size=None),
          error_score='raise-deprecating',
          estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=0, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=10, n_jobs=None,
          param_distributions={'loss': ['hinge', 'log', 'squared_hinge', 'squared_loss'], 'penalty': ['l1', 'l2'], 'max_iter': range(5, 10), 'alpha': array([0.0001 , 0.00032, 0.00055, 0.00078, 0.001  ])},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=0)

In [128]:
print (randomized_grid_cv.best_score_)
print (randomized_grid_cv.best_params_)

0.8666666666666667
{'penalty': 'l1', 'max_iter': 9, 'loss': 'log', 'alpha': 0.00055}
